In [ ]:
%matplotlib inline
%load_ext Cython

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import pickle
import librosa as lb
import librosa.display
import time
import scipy
import seaborn as sns
import os
import multiprocessing
from scipy.spatial.distance import cdist
import pandas as pd

In [ ]:
%%cython
import numpy as np
cimport numpy as np
cimport cython

import sys
import time


DTYPE_INT32 = np.int32
ctypedef np.int32_t DTYPE_INT32_t

DTYPE_FLOAT = np.float64
ctypedef np.float64_t DTYPE_FLOAT_t

cdef DTYPE_FLOAT_t MAX_FLOAT = float('inf')

# careful, without bounds checking can mess up memory - also can't use negative indices I think (like x[-1])
@cython.boundscheck(False) # turn off bounds-checking for entire function
def DTW_Cost_To_AccumCostAndSteps(Cin, parameter):
    '''
    Inputs
        C: The cost Matrix
    '''


    '''
    Section for checking and catching errors in the inputs
    '''

    cdef np.ndarray[DTYPE_FLOAT_t, ndim=2] C
    try:
        C = np.array(Cin, dtype=DTYPE_FLOAT)
    except TypeError:
        print(bcolors.FAIL + "FAILURE: The type of the cost matrix is wrong - please pass in a 2-d numpy array" + bcolors.ENDC)
        return [-1, -1, -1]
    except ValueError:
        print(bcolors.FAIL + "FAILURE: The type of the elements in the cost matrix is wrong - please have each element be a float (perhaps you passed in a matrix of ints?)" + bcolors.ENDC)
        return [-1, -1, -1]

    cdef np.ndarray[np.uint32_t, ndim=1] dn
    cdef np.ndarray[np.uint32_t, ndim=1] dm
    cdef np.ndarray[DTYPE_FLOAT_t, ndim=1] dw
    # make sure dn, dm, and dw are setup
    # dn loading and exception handling
    if ('dn'  in parameter.keys()):
        try:

            dn = np.array(parameter['dn'], dtype=np.uint32)
        except TypeError:
            print(bcolors.FAIL + "FAILURE: The type of dn (row steps) is wrong - please pass in a 1-d numpy array that holds uint32s" + bcolors.ENDC)
            return [-1, -1, -1]
        except ValueError:
            print(bcolors.FAIL + "The type of the elements in dn (row steps) is wrong - please have each element be a uint32 (perhaps you passed a long?). You can specify this when making a numpy array like: np.array([1,2,3],dtype=np.uint32)" + bcolors.ENDC)
            return [-1, -1, -1]
    else:
        dn = np.array([1, 1, 0], dtype=np.uint32)
    # dm loading and exception handling
    if 'dm'  in parameter.keys():
        try:
            dm = np.array(parameter['dm'], dtype=np.uint32)
        except TypeError:
            print(bcolors.FAIL + "FAILURE: The type of dm (col steps) is wrong - please pass in a 1-d numpy array that holds uint32s" + bcolors.ENDC)
            return [-1, -1, -1]
        except ValueError:
            print(bcolors.FAIL + "FAILURE: The type of the elements in dm (col steps) is wrong - please have each element be a uint32 (perhaps you passed a long?). You can specify this when making a numpy array like: np.array([1,2,3],dtype=np.uint32)" + bcolors.ENDC)
            return [-1, -1, -1]
    else:
        print(bcolors.FAIL + "dm (col steps) was not passed in (gave default value [1,0,1]) " + bcolors.ENDC)
        dm = np.array([1, 0, 1], dtype=np.uint32)
    # dw loading and exception handling
    if 'dw'  in parameter.keys():
        try:
            dw = np.array(parameter['dw'], dtype=DTYPE_FLOAT)
        except TypeError:
            print(bcolors.FAIL + "FAILURE: The type of dw (step weights) is wrong - please pass in a 1-d numpy array that holds floats" + bcolors.ENDC)
            return [-1, -1, -1]
        except ValueError:
            print(bcolors.FAIL + "FAILURE:The type of the elements in dw (step weights) is wrong - please have each element be a float (perhaps you passed ints or a long?). You can specify this when making a numpy array like: np.array([1,2,3],dtype=np.float64)" + bcolors.ENDC)
            return [-1, -1, -1]
    else:
        dw = np.array([1, 1, 1], dtype=DTYPE_FLOAT)
        print(bcolors.FAIL + "dw (step weights) was not passed in (gave default value [1,1,1]) " + bcolors.ENDC)

    
    '''
    Section where types are given to the variables we're going to use 
    '''
    # create matrices to store our results (D and E)
    cdef DTYPE_INT32_t numRows = C.shape[0] # only works with np arrays, use np.shape(x) will work on lists? want to force to use np though?
    cdef DTYPE_INT32_t numCols = C.shape[1]
    cdef DTYPE_INT32_t numDifSteps = np.size(dw)

    cdef unsigned int maxRowStep = max(dn)
    cdef unsigned int maxColStep = max(dm)

    cdef np.ndarray[np.uint32_t, ndim=2] steps = np.zeros((numRows,numCols), dtype=np.uint32)
    cdef np.ndarray[DTYPE_FLOAT_t, ndim=2] accumCost = np.ones((maxRowStep + numRows, maxColStep + numCols), dtype=DTYPE_FLOAT) * MAX_FLOAT

    cdef DTYPE_FLOAT_t bestCost
    cdef DTYPE_INT32_t bestCostIndex
    cdef DTYPE_FLOAT_t costForStep
    cdef unsigned int row, col
    cdef unsigned int stepIndex

    '''
    The start of the actual algorithm, now that all our variables are set up
    '''
    # initializing the cost matrix - depends on whether its subsequence DTW
    # essentially allow us to hop on the bottom anywhere (so could start partway through one of the signals)
    if parameter['SubSequence']:
        for col in range(numCols):
            accumCost[maxRowStep, col + maxColStep] = C[0, col]
    else:
        accumCost[maxRowStep, maxColStep] = C[0,0]

    # filling the accumulated cost matrix
    for row in range(maxRowStep, numRows + maxRowStep, 1):
        for col in range(maxColStep, numCols + maxColStep, 1):
            bestCost = accumCost[<unsigned int>row, <unsigned int>col] # initialize with what's there - so if is an entry point, then can start low
            bestCostIndex = 0
            # go through each step, find the best one
            for stepIndex in range(numDifSteps):
                #costForStep = accumCost[<unsigned int>(row - dn[<unsigned int>(stepIndex)]), <unsigned int>(col - dm[<unsigned int>(stepIndex)])] + dw[<unsigned int>(stepIndex)] * C[<unsigned int>(row - maxRowStep), <unsigned int>(col - maxColStep)]
                costForStep = accumCost[<unsigned int>((row - dn[(stepIndex)])), <unsigned int>((col - dm[(stepIndex)]))] + dw[stepIndex] * C[<unsigned int>(row - maxRowStep), <unsigned int>(col - maxColStep)]
                if costForStep < bestCost:
                    bestCost = costForStep
                    bestCostIndex = stepIndex
            # save the best cost and best cost index
            accumCost[row, col] = bestCost
            steps[<unsigned int>(row - maxRowStep), <unsigned int>(col - maxColStep)] = bestCostIndex

    # return the accumulated cost along with the matrix of steps taken to achieve that cost
    return [accumCost[maxRowStep:, maxColStep:], steps]

@cython.boundscheck(False) # turn off bounds-checking for entire function
def DTW_GetPath(np.ndarray[DTYPE_FLOAT_t, ndim=2] accumCost, np.ndarray[np.uint32_t, ndim=2] stepsForCost, parameter):
    '''

    Parameter should have: 'dn', 'dm', 'dw', 'SubSequence'
    '''

    cdef np.ndarray[unsigned int, ndim=1] dn
    cdef np.ndarray[unsigned int, ndim=1] dm
    cdef np.uint8_t subseq
    # make sure dn, dm, and dw are setup
    if ('dn'  in parameter.keys()):
        dn = parameter['dn']
    else:
        dn = np.array([1, 1, 0], dtype=DTYPE_INT32)
    if 'dm'  in parameter.keys():
        dm = parameter['dm']
    else:
        dm = np.array([1, 0, 1], dtype=DTYPE_INT32)
    if 'SubSequence' in parameter.keys():
        subseq = parameter['SubSequence']
    else:
        subseq = 0

    cdef np.uint32_t numRows
    cdef np.uint32_t numCols
    cdef np.uint32_t curRow
    cdef np.uint32_t curCol
    cdef np.uint32_t endCol
    cdef DTYPE_FLOAT_t endCost

    numRows = accumCost.shape[0]
    numCols = accumCost.shape[1]

    # either start at the far corner (non sub-sequence)
    # or start at the lowest cost entry in the last row (sub-sequence)
    # where all of the signal along the row has been used, but only a 
    # sub-sequence of the signal along the columns has to be used
    curRow = numRows - 1
    if subseq:
        curCol = np.argmin(accumCost[numRows - 1, :])
    else:
        curCol = numCols - 1

    endCol = curCol
    endCost = accumCost[curRow, curCol]

    cdef np.uint32_t curRowStep
    cdef np.uint32_t curColStep
    cdef np.uint32_t curStepIndex


    cdef np.ndarray[np.uint32_t, ndim=2] path = np.zeros((2, numRows + numCols), dtype=np.uint32) # make as large as could need, then chop at the end
    path[0, 0] = curRow
    path[1, 0] = curCol

    cdef np.uint32_t stepsInPath = 1 # starts at one, we add in one before looping
    cdef np.uint32_t stepIndex = 0
    cdef np.int8_t done = (subseq and curRow == 0) or (curRow == 0 and curCol == 0)
    while not done:
        if accumCost[curRow, curCol] == MAX_FLOAT:
            print('A path is not possible')
            break

        # you're done if you've made it to the bottom left (non sub-sequence)
        # or just the bottom (sub-sequence)
        # find the step size
        curStepIndex = stepsForCost[curRow, curCol]
        curRowStep = dn[curStepIndex]
        curColStep = dm[curStepIndex]
        # backtrack by 1 step
        curRow = curRow - curRowStep
        curCol = curCol - curColStep
        # add your new location onto the path
        path[0, stepsInPath] = curRow
        path[1, stepsInPath] = curCol
        stepsInPath = stepsInPath + 1
        # check to see if you're done
        done = (subseq and curRow == 0) or (curRow == 0 and curCol == 0)

    # reverse the path (a matrix with two rows) and return it
    return [np.fliplr(path[:, 0:stepsInPath]), endCol, endCost]

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [ ]:
def cosine_distance(X,Y):
    cost = cdist(X,Y,'cosine')
    return cost

In [ ]:
def align_chromas(query, ref, steps = [1,1,1,2,2,1], weights = [2, 3, 3], optimized = True):
    if optimized:
        # set params
        assert len(steps) % 2 == 0, "The length of steps must be even."
        dn = np.array(steps[::2], dtype=np.uint32)
        dm = np.array(steps[1::2], dtype=np.uint32)
        dw = weights
        subsequence = True
        parameter = {'dn': dn, 'dm': dm, 'dw': dw, 'SubSequence': subsequence}

        # Compute cost matrix
        cost = cosine_distance(query.T, ref.T)

        # DTW
        [D, s] = DTW_Cost_To_AccumCostAndSteps(cost, parameter)
        [wp, endCol, endCost] = DTW_GetPath(D, s, parameter)

        # Reformat the output
        wp = wp.T[::-1]
    else: # librosa implementation
        steps = np.array(steps).reshape((-1,2))
        D, wp = lb.sequence.dtw(query, ref, step_sizes_sigma = steps, weights_mul = weights, subseq = True, metric = 'cosine')
    return D, wp

In [ ]:
def get_chroma_features(audio,sr=22050,H=512,N=2048, cens=False):
    if not cens:
        chroma = lb.feature.chroma_stft(y=audio, sr=sr, norm=2, hop_length=H, n_fft=N)
    else:
        chroma = lb.feature.chroma_cens(y=audio, sr=sr, norm=2, hop_length=H, n_fft=N)
    return chroma

In [ ]:
def get_audio(path):
    audio, sr = lb.core.load(path)
    return audio

In [ ]:
piano3_audio = get_audio('data/Audio/piano/piano3.m4a')
violin3_audio = get_audio('data/Audio/violin/violin3.mp3')
cello3_audio = get_audio('data/Audio/cello/cello3.mp3')
piano4_audio = get_audio("data/Audio/piano/piano4.m4a")
violin4_audio = get_audio('data/Audio/violin/violin4.mp3')
cello4_audio = get_audio('data/Audio/cello/cello4.mp3')
fullmix_audio_3 = get_audio('data/Audio/fullmix/fullmix3.mp3')
fullmix_audio_4 = get_audio('data/Audio/fullmix/fullmix4.mp3')

### Generate chroma features

In [ ]:
piano3_chroma = get_chroma_features(piano3_audio)
violin3_chroma = get_chroma_features(violin3_audio)
cello3_chroma = get_chroma_features(cello3_audio)
piano4_chroma = get_chroma_features(piano4_audio)
violin4_chroma = get_chroma_features(violin4_audio)
cello4_chroma = get_chroma_features(cello4_audio)
fullmix_chroma_3 = get_chroma_features(fullmix_audio_3)
fullmix_chroma_4 = get_chroma_features(fullmix_audio_4)

In [ ]:
idx = np.where(~fullmix_chroma_3.any(axis=0))[0]

In [ ]:
padding = np.full((fullmix_chroma_3.shape[0]), 1e-9)

In [ ]:
for i in idx:
    fullmix_chroma_3[:, i] = padding

In [ ]:
idx = np.where(~fullmix_chroma_4.any(axis=0))[0]

In [ ]:
padding = np.full((fullmix_chroma_4.shape[0]), 1e-9)

In [ ]:
for i in idx:
    fullmix_chroma_4[:, i] = padding

### Align chroma features using DTW

In [ ]:
piano3_fullmix_3 = align_chromas(piano3_chroma, fullmix_chroma_3)
violin3_fullmix_3 = align_chromas(violin3_chroma, fullmix_chroma_3)
cello3_fullmix_3 = align_chromas(cello3_chroma, fullmix_chroma_3)

In [ ]:
piano3_fullmix_4 = align_chromas(piano3_chroma, fullmix_chroma_4)
violin3_fullmix_4 = align_chromas(violin3_chroma, fullmix_chroma_4)
cello3_fullmix_4 = align_chromas(cello3_chroma, fullmix_chroma_4)

In [ ]:
piano4_fullmix_3 = align_chromas(piano4_chroma, fullmix_chroma_3)
violin4_fullmix_3 = align_chromas(violin4_chroma, fullmix_chroma_3)
cello4_fullmix_3 = align_chromas(cello4_chroma, fullmix_chroma_3)

In [ ]:
piano4_fullmix_4 = align_chromas(piano4_chroma, fullmix_chroma_4)
violin4_fullmix_4 = align_chromas(violin4_chroma, fullmix_chroma_4)
cello4_fullmix_4 = align_chromas(cello4_chroma, fullmix_chroma_4)

In [ ]:
fullmix_fullmix = align_chromas(fullmix_chroma_3, fullmix_chroma_4)

In [ ]:
def frame_to_time(frame, hop_length = 512, sr = 22050):
    return frame * hop_length / sr

In [ ]:
def time_to_frame(time, hop_length = 512, sr = 22050):
    return time * sr / hop_length

In [ ]:
def calculateErrors(data, annotfile, gtfile, hop_length = 512, sr = 22050, debug = False):
    wp = np.array(sorted(data, key = lambda x: x[0]))
    query_preds = wp[:, 0]
    ref_preds = wp[:, 1]
    query_to_ref = np.interp(list(range(max(query_preds[-1], ref_preds[-1]) + 1)), query_preds, ref_preds)

    data_gt = np.genfromtxt(gtfile, delimiter=',')
    gt_mapping = {}
    for time, idx in data_gt:
        gt_mapping[idx] = time
        
    data_annot = np.genfromtxt(annotfile, delimiter=',')
    errors = []
    clicks = []
    idxs = []
    for time, idx in data_annot:
        frame = int(np.round(time_to_frame(time)))
        ref_frame = query_to_ref[frame]
        pred_ref_time = frame_to_time(ref_frame)
        if debug:
            clicks.append(pred_ref_time)
        if idx in gt_mapping:
            error = np.abs(pred_ref_time - gt_mapping[idx])
            idxs.append(idx)
            errors.append(error)
    if debug:
        return errors, clicks
    return errors

In [ ]:
annot_path = './data/Annotations/merged/merged/'

piano3_fullmix_3_errors = calculateErrors(piano3_fullmix_3[1], annot_path+'piano3.csv', annot_path+'fullmix3.csv')
violin3_fullmix_3_errors = calculateErrors(violin3_fullmix_3[1], annot_path+'violin3.csv', annot_path+'fullmix3.csv')
cello3_fullmix_3_errors = calculateErrors(cello3_fullmix_3[1], annot_path+'cello3.csv', annot_path+'fullmix3.csv')

piano4_fullmix_3_errors = calculateErrors(piano4_fullmix_3[1], annot_path+'piano4.csv', annot_path+'fullmix3.csv')
violin4_fullmix_3_errors = calculateErrors(violin4_fullmix_3[1], annot_path+'violin4.csv', annot_path+'fullmix3.csv')
cello4_fullmix_3_errors = calculateErrors(cello4_fullmix_3[1], annot_path+'cello4.csv', annot_path+'fullmix3.csv')

piano3_fullmix_4_errors = calculateErrors(piano3_fullmix_4[1], annot_path+'piano3.csv', annot_path+'fullmix4.csv')
violin3_fullmix_4_errors = calculateErrors(violin3_fullmix_4[1], annot_path+'violin3.csv', annot_path+'fullmix4.csv')
cello3_fullmix_4_errors = calculateErrors(cello3_fullmix_4[1], annot_path+'cello3.csv', annot_path+'fullmix4.csv')

piano4_fullmix_4_errors = calculateErrors(piano4_fullmix_4[1], annot_path+'piano4.csv', annot_path+'fullmix4.csv')
violin4_fullmix_4_errors = calculateErrors(violin4_fullmix_4[1], annot_path+'violin4.csv', annot_path+'fullmix4.csv')
cello4_fullmix_4_errors = calculateErrors(cello4_fullmix_4[1], annot_path+'cello4.csv', annot_path+'fullmix4.csv')

fullmixes_errors = calculateErrors(fullmix_fullmix[1], annot_path+'fullmix3.csv', annot_path+'fullmix4.csv')

In [ ]:
def get_tolerances(errors, tols):
    errors = np.array(errors)
    errorRates = []
    for tol in tols:
        toAdd = np.sum(errors > tol) * 1.0 / len(errors)
        errorRates.append(toAdd)
    return errorRates

In [ ]:
piano3_errs_3 = get_tolerances(piano3_fullmix_3_errors, np.arange(0,1,1/1000))
violin3_errs_3 = get_tolerances(violin3_fullmix_3_errors, np.arange(0,1,1/1000))
cello3_errs_3 = get_tolerances(cello3_fullmix_3_errors, np.arange(0,1,1/1000))
piano3_errs_4 = get_tolerances(piano3_fullmix_4_errors, np.arange(0,1,1/1000))
violin3_errs_4 = get_tolerances(violin3_fullmix_4_errors, np.arange(0,1,1/1000))
cello3_errs_4 = get_tolerances(cello3_fullmix_4_errors, np.arange(0,1,1/1000))

piano4_errs_3 = get_tolerances(piano4_fullmix_3_errors, np.arange(0,1,1/1000))
violin4_errs_3 = get_tolerances(violin4_fullmix_3_errors, np.arange(0,1,1/1000))
cello4_errs_3 = get_tolerances(cello4_fullmix_3_errors, np.arange(0,1,1/1000))
piano4_errs_4 = get_tolerances(piano4_fullmix_4_errors, np.arange(0,1,1/1000))
violin4_errs_4 = get_tolerances(violin4_fullmix_4_errors, np.arange(0,1,1/1000))
cello4_errs_4 = get_tolerances(cello4_fullmix_4_errors, np.arange(0,1,1/1000))

fullmix_errs = get_tolerances(fullmixes_errors, np.arange(0,1,1/1000))

In [ ]:
np.save('./DTW-Chroma/tolerances/piano/piano3_errs_3',piano3_errs_3)
np.save('./DTW-Chroma/tolerances/violin/violin3_errs_3',violin3_errs_3)
np.save('./DTW-Chroma/tolerances/cello/cello3_errs_3',cello3_errs_3)
np.save('./DTW-Chroma/tolerances/piano/piano3_errs_4',piano3_errs_4)
np.save('./DTW-Chroma/tolerances/violin/violin3_errs_4',violin3_errs_4)
np.save('./DTW-Chroma/tolerances/cello/cello3_errs_4',cello3_errs_4)

np.save('./DTW-Chroma/tolerances/piano/piano4_errs_3',piano4_errs_3)
np.save('./DTW-Chroma/tolerances/violin/violin4_errs_3',violin4_errs_3)
np.save('./DTW-Chroma/tolerances/cello/cello4_errs_3',cello4_errs_3)
np.save('./DTW-Chroma/tolerances/piano/piano4_errs_4',piano4_errs_4)
np.save('./DTW-Chroma/tolerances/violin/violin4_errs_4',violin4_errs_4)
np.save('./DTW-Chroma/tolerances/cello/cello4_errs_4',cello4_errs_4)

np.save('./DTW-Chroma/tolerances/fullmix_errs',fullmix_errs)

In [ ]:
piano_errs_1 = np.mean([piano3_errs_3, piano4_errs_3], axis=0)
piano_errs_2 = np.mean([piano3_errs_4, piano4_errs_4], axis=0)
violin_errs_1 = np.mean([violin3_errs_3, violin4_errs_3], axis=0)
violin_errs_2 = np.mean([violin3_errs_4, violin4_errs_4], axis=0)
cello_errs_1 = np.mean([cello3_errs_3, cello4_errs_3], axis=0)
cello_errs_2 = np.mean([cello3_errs_4, cello4_errs_4], axis=0)

In [ ]:
data = {'piano-fullmix1': piano_errs_1, 'violin-fullmix1': violin_errs_1, 'cello-fullmix1': cello_errs_1, 'piano-fullmix2': piano_errs_2,
        'violin-fullmix2': violin_errs_2, 'cello-fullmix2': cello_errs_2, 'fullmix1-fullmix2': fullmix_errs}

In [ ]:
df = pd.DataFrame.from_dict(data)

In [ ]:
df.head()

In [ ]:
plt.figure(figsize = (10,10))
sns.set_style("whitegrid")
sns.lineplot(data = df, dashes=False)
plt.ylabel('Error Rate (%)')
plt.xlabel('Tolerance (ms)')
plt.title('Error Rate vs. Tolerance - subsequence DTW')